<a href="https://colab.research.google.com/github/nishant3399/neural-network-challenge-2/blob/main/attrition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part 1: Preprocessing

In [8]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [9]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [10]:
# Create y_df with the Attrition and Department columns

y_df = attrition_df[['Attrition', 'Department']]

In [11]:
# Create a list of at least 10 column names to use as X data

selected_columns = [
    'Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
    'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'NumCompaniesWorked',
    'TotalWorkingYears', 'TrainingTimesLastYear'
]

# Create X_df using your selected columns

X_df = attrition_df[selected_columns]

# Show the data types for X_df

print(X_df.dtypes)

Age                        int64
DistanceFromHome           int64
Education                  int64
EnvironmentSatisfaction    int64
JobInvolvement             int64
JobLevel                   int64
JobSatisfaction            int64
NumCompaniesWorked         int64
TotalWorkingYears          int64
TrainingTimesLastYear      int64
dtype: object


In [16]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)

In [17]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
categorical_columns = X_df.select_dtypes(include=['object']).columns.tolist()

In [18]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [20]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data

y_train_department_encoded = department_encoder.transform(y_train[['Department']])
y_test_department_encoded = department_encoder.transform(y_test[['Department']])


In [21]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)


# Fit the encoder to the training data

attrition_encoder.fit(y_train[['Attrition']])
# Create two new variables by applying the encoder
# to the training and testing data

y_train_attrition_encoded = attrition_encoder.transform(y_train[['Attrition']])
y_test_attrition_encoded = attrition_encoder.transform(y_test[['Attrition']])

## Create, Compile, and Train the Model

In [24]:
# Find the number of columns in the X training data
input_shape = X_train_scaled.shape[1]
print(f"Number of columns in the X training data: {input_shape}")

# Create the input layer
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

input_layer = Input(shape=(input_shape,))

# Create at least two shared layers
shared_layer_1 = Dense(64, activation='relu')(input_layer)
shared_layer_2 = Dense(32, activation='relu')(shared_layer_1)

Number of columns in the X training data: 10


In [26]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_branch_hidden_layer = Dense(16, activation='relu')(shared_layer_2)


# Create the output layer

department_output_layer = Dense(y_train_department_encoded.shape[1], activation='softmax', name='department_output')(department_branch_hidden_layer)

In [27]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_branch_hidden_layer = Dense(16, activation='relu')(shared_layer_2)

# Create the output layer
attrition_output_layer = Dense(y_train_attrition_encoded.shape[1], activation='softmax', name='attrition_output')(attrition_branch_hidden_layer)

In [28]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output_layer, attrition_output_layer])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy', 'attrition_output': 'categorical_crossentropy'},
              metrics={'department_output': 'accuracy', 'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 10)]                 0         []                            
                                                                                                  
 dense_2 (Dense)             (None, 64)                   704       ['input_2[0][0]']             
                                                                                                  
 dense_3 (Dense)             (None, 32)                   2080      ['dense_2[0][0]']             
                                                                                                  
 dense_5 (Dense)             (None, 16)                   528       ['dense_3[0][0]']             
                                                                                              

In [29]:


# Train the model
history = model.fit(
    X_train_scaled,
    {'department_output': y_train_department_encoded, 'attrition_output': y_train_attrition_encoded},
    epochs=50,
    batch_size=32,
    validation_split=0.2
)

Epoch 1/50
30/30 [==============================] - 2s 18ms/step - loss: 1.4231 - department_output_loss: 0.9602 - attrition_output_loss: 0.4629 - department_output_accuracy: 0.5702 - attrition_output_accuracy: 0.8404 - val_loss: 1.4014 - val_department_output_loss: 0.8906 - val_attrition_output_loss: 0.5108 - val_department_output_accuracy: 0.6314 - val_attrition_output_accuracy: 0.7966
Epoch 2/50
30/30 [==============================] - 0s 4ms/step - loss: 1.2370 - department_output_loss: 0.8071 - attrition_output_loss: 0.4299 - department_output_accuracy: 0.6564 - attrition_output_accuracy: 0.8404 - val_loss: 1.3320 - val_department_output_loss: 0.8243 - val_attrition_output_loss: 0.5077 - val_department_output_accuracy: 0.6144 - val_attrition_output_accuracy: 0.7966
Epoch 3/50
30/30 [==============================] - 0s 4ms/step - loss: 1.1837 - department_output_loss: 0.7738 - attrition_output_loss: 0.4100 - department_output_accuracy: 0.6500 - attrition_output_accuracy: 0.8426 - 

In [30]:
# Evaluate the model with the testing data
evaluation = model.evaluate(
    X_test_scaled,
    {'department_output': y_test_department_encoded, 'attrition_output': y_test_attrition_encoded}
)

# Print the evaluation results
print(f"Department Loss: {evaluation[1]}, Department Accuracy: {evaluation[3]}")
print(f"Attrition Loss: {evaluation[2]}, Attrition Accuracy: {evaluation[4]}")

10/10 [==============================] - 0s 6ms/step - loss: 1.3037 - department_output_loss: 0.8904 - attrition_output_loss: 0.4133 - department_output_accuracy: 0.6293 - attrition_output_accuracy: 0.8673
Department Loss: 0.8904119729995728, Department Accuracy: 0.6292517185211182
Attrition Loss: 0.41327401995658875, Attrition Accuracy: 0.8673469424247742


In [31]:
# Print the accuracy for both department and attrition
print(f"Department Accuracy: {evaluation[3]}")
print(f"Attrition Accuracy: {evaluation[4]}")

Department Accuracy: 0.6292517185211182
Attrition Accuracy: 0.8673469424247742


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy may not be the best metric if the classes are imbalanced, as it can be misleading; metrics such as precision, recall, or F1-score might provide better insights into model performance.
2. I chose the softmax activation function for the output layers because it is appropriate for multi-class classification problems, providing probabilities for each class.
3. The model could be improved by tuning hyperparameters, adding regularization techniques to prevent overfitting, or using more advanced neural network architectures.